In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [68]:
import pandas as pd
from pathlib import Path
import numpyro
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from src import preprocessing
from scipy import stats

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [3]:
raw_data = pd.read_csv(path_raw_data / "State_2020.csv")
# data = preprocessing.preprocess(raw_data)
# raw_data["county_fips"] = raw_data["county_fips"].astype(int)

/tmp/ipykernel_1649735/2796876182.py:1: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "State_2020.csv")


In [4]:
group_cols = ["state", "district", "office", "candidate", "party_simplified"]
vote_totals = raw_data[[*group_cols, "votes"]].groupby(group_cols).sum()

In [8]:
group_cols = ["county_fips", "state", "district", "office"]
df = raw_data[[*group_cols, "votes"]].groupby(group_cols).sum()
df = df.reset_index(level=[1,2,3])
df["election_name"] = df["state"].astype(str) + "|" + df["district"].astype(str) + "|" + df["office"].astype(str)
county_to_election_name = {}

for key, row in df.iterrows():
  if key in county_to_election_name:
    county_to_election_name[key].append(row["election_name"])
  else:
    county_to_election_name[key] = [row["election_name"]]

election_names = df["election_name"].unique()

In [29]:
county_parties = raw_data[["county_fips", "party_simplified"]].groupby(["county_fips", "party_simplified"]).count().reset_index()
# county_to_election_name
partisan_election = (county_parties["party_simplified"] == "REPUBLICAN") | (county_parties["party_simplified"] == "DEMOCRAT")
county_competitive_partisan = (county_parties[partisan_election].groupby("county_fips").nunique() == 2)
competitive_partisan_counties = county_competitive_partisan[county_competitive_partisan["party_simplified"]].index


In [7]:
election_name = county_to_election_name[12001.0][0]
election_index = tuple(election_name.split("|"))
print(election_name)
vote_totals.loc[election_index]

FLORIDA|010|STATE HOUSE


,,votes
candidate,party_simplified,
CHUCK BRANNAN,REPUBLICAN,58872
ROCK ABOUJAOUDE JR,DEMOCRAT,16456


In [11]:
election_indices = [tuple(election_name.split("|")) for election_name in election_names]

In [40]:
election_names[0]

'ALABAMA|003|STATE BOARD OF EDUCATION'

In [12]:
import numpyro
from numpyro import distributions as dist

In [54]:
# vote_totals.loc[election_indices[2]]
df = vote_totals.reset_index()
df["election_name"] = df["state"] + "|" + df["district"].astype(str) + "|" + df["office"]
df[["election_name", "party_simplified", "votes"]].pivot_table(values="votes", columns="party_simplified", index="election_name")
# def model()

party_simplified,DEMOCRAT,LIBERTARIAN,NONPARTISAN,OTHER,REPUBLICAN
election_name,,,,,
ALABAMA|001|STATE BOARD OF EDUCATION,79380.0,NaN,NaN,NaN,212461.0
ALABAMA|003|STATE BOARD OF EDUCATION,94375.0,NaN,NaN,NaN,209909.0
ALABAMA|005|STATE BOARD OF EDUCATION,161192.0,NaN,NaN,NaN,95593.0
ALABAMA|007|STATE BOARD OF EDUCATION,NaN,NaN,NaN,NaN,230122.0
"ALABAMA|CIRCUIT 10, PLACE 10|CIRCUIT COURT JUDGE",195110.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
WYOMING|058|STATE HOUSE,NaN,646.0,NaN,NaN,2725.0
WYOMING|059|STATE HOUSE,1104.0,NaN,NaN,NaN,2658.0
WYOMING|060|STATE HOUSE,1260.0,NaN,NaN,NaN,2793.0


In [7]:
group_cols = ["county_fips", "state", "district", "office", "candidate", "party_simplified"]
vote_totals = raw_data[[*group_cols, "votes"]].groupby(group_cols).sum()

In [62]:
county_partisan = (
    vote_totals.reset_index(-1)
    .query("party_simplified == 'DEMOCRAT' or party_simplified == 'REPUBLICAN'")
    .pivot_table(
        values="votes",
        index=["county_fips", "district", "office"],
        columns=["party_simplified"],
        aggfunc="sum",
    )
)

county_partisan["total_votes"] = (
    county_partisan["DEMOCRAT"] + county_partisan["REPUBLICAN"]
)

county_partisan["p_democrat"] = (
    county_partisan["DEMOCRAT"] / county_partisan["total_votes"]
)

county_partisan_agg = county_partisan.dropna().reset_index().groupby("county_fips")

county_probs = pd.DataFrame(
    {
        "mean_p_democrat": county_partisan_agg.mean(numeric_only=True)["p_democrat"],
        "std_p_democrat": county_partisan_agg.std(numeric_only=True)["p_democrat"],
        "obs_p_democrat": county_partisan_agg.count()["p_democrat"],
        "total_votes": county_partisan_agg.max(numeric_only=True)["total_votes"]
    }
)

county_probs["std_p_democrat"] = county_probs["std_p_democrat"].fillna(
    county_probs["std_p_democrat"].dropna().mean()
)

In [69]:
county_probs
for ind, row in county_probs.iterrows():
    mean_p = row["mean_p_democrat"]
    std_p = row["std_p_democrat"]
    county_probs.loc[ind, "p_05"] = stats.t.pdf(0.5, 4, loc=mean_p, scale=std_p)
county_probs["pivot_odds"] = county_probs["p_05"] / county_probs["total_votes"]
county_probs["log_pivot_odds"] = np.log(county_probs["p_05"] / county_probs["total_votes"])

In [70]:
to_save = county_probs.reset_index()[["county_fips", "pivot_odds", "log_pivot_odds"]]

In [72]:
to_save.to_csv(path_data / "partisan_county_pivot_odds.csv")